In [1]:
import Pkg

Pkg.add(url="https://github.com/matthiasbaitsch/mmjmesh.git")

include("setup.jl")

    Updating git-repo `https://github.com/matthiasbaitsch/mmjmesh.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [010ee2c9] ~ MMJMesh v0.1.1 `https://github.com/matthiasbaitsch/mmjmesh.git#main` ⇒ v0.1.1 `https://github.com/matthiasbaitsch/mmjmesh.git#main`
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [010ee2c9] ~ MMJMesh v0.1.1 `https://github.com/matthiasbaitsch/mmjmesh.git#main` ⇒ v0.1.1 `https://github.com/matthiasbaitsch/mmjmesh.git#main`
Precompiling project...
  ✓ MMJMesh
  1 dependency successfully precompiled in 15 seconds. 416 already precompiled.


3.1e10

In [13]:
include("weak_form_hartmann.jl")

Ke[1,1] = Ke[4,4] = Ke[7,7] = Ke[10,10]
             = (20E*(a^4)*(h^3) + 7E*(a^2)*(b^2)*(h^3) + 20E*(b^4)*(h^3) + 3E*(a^2)*(b^2)*(h^3)*ν) / (60(a^3)*(b^3)*(1 - (ν^2)))

Ke[1,2] = Ke[2,1] = Ke[10,11] = Ke[11,10]
             = (E*(a^2)*(h^3) + 20E*(b^2)*(h^3) + 9E*(a^2)*(h^3)*ν) / (120(a^2)*b*(1 - (ν^2)))

Ke[1,3] = Ke[3,1] = Ke[4,6] = Ke[6,4]
             = (20E*(a^2)*(h^3) + E*(b^2)*(h^3) + 9E*(b^2)*(h^3)*ν) / (120a*(b^2)*(1 - (ν^2)))

Ke[1,4] = Ke[4,1] = Ke[7,10] = Ke[10,7]
             = (10E*(a^4)*(h^3) - 7E*(a^2)*(b^2)*(h^3) - 20E*(b^4)*(h^3) - 3E*(a^2)*(b^2)*(h^3)*ν) / (60(a^3)*(b^3)*(1 - (ν^2)))

Ke[1,5] = Ke[5,1] = Ke[8,10] = Ke[10,8]
             = (E*(a^2)*(h^3) + 20E*(b^2)*(h^3) - E*(a^2)*(h^3)*ν) / (120(a^2)*b*(1 - (ν^2)))

Ke[1,6] = Ke[3,4] = Ke[4,3] = Ke[6,1]
             = (10E*(a^2)*(h^3) - E*(b^2)*(h^3) - 9E*(b^2)*(h^3)*ν) / (120a*(b^2)*(1 - (ν^2)))

Ke[1,7] = Ke[4,10] = Ke[7,1] = Ke[10,4]
             = (-10E*(a^4)*(h^3) + 7E*(a^2)*(b^2)*(h^3) - 10E*(b^4)*(h^3) + 3E*

In [2]:
dofs = zeros(40)      
ma = zeros(40)

for i = 1:40
        m,w = plate(p, 2*i)     # nx = ny = Anzahl der Elemente in x- bzw y-Richtung 
        # DOFs
        bN = m.groups[:boundarynodes]
        dofs[i] = nnodes(m) * 3 -  size(bN,1) * 3
        # max w
        u = zeros(trunc(Int,(size(w,1)/3)))
        for i = 1:(trunc(Int,(size(w,1)/3)))
                u[i] = w[i*3-2]
        end
        ma[i] = maximum(w)
end

In [9]:
using Plots
Plots.plot(dofs, ma*1000)
Plots.plot!(title = "Konvergenzplot", xlabel = "Anzahl Freheitsgrade", ylabel = "maximale Verformung [mm]")
Plots.plot!(xlims = (0, 4000), ylims = (1.25, 1.35), xticks = 0:1000:4000, yticks = 1.26:0.02:1.34);

In [10]:
plotsol(p,5)

UndefVarError: UndefVarError: `_fsize` not defined

In [12]:
m,wHat = plate(p,5)
postprocessor(p, wHat)

#109 (generic function with 1 method)

In [18]:
function valuerange(p)
    values = p.plots[1].kw[:color]
    min, max = extrema(values)
    return min, max
end
valuerange(p)

KeyError: KeyError: key :plots not found

In [14]:
function maketitle(p, title)
    min, max = string.(round.(valuerange(p), digits=2))
    return title * " | min: " * min * " | max: " * max
end

function plotr(m, result, title, cr, npoints=15; nodal=false, a3d)
    fig = Figure(size=(1200, 800))
    if a3d
        ax = Axis3(fig[1, 1])
        zscale = 1
    else
        ax = Axis(fig[1, 1], aspect=DataAspect())
        zscale = 0
    end

    warpnodes=nothing

    if nodal
        r = nodalresult(m, result)
        if a3d
          warpnodes=r
        end
    else
        r = result
    end

    p = mplot!(
        m, r, edgesvisible=a3d,
        nodewarp=warpnodes,
        faceplotzscale=zscale, faceplotnpoints=npoints,
        colorrange=cr
    )

    if !a3d
        mplot!(m, edgesvisible=false, facesvisible=false)
    end

    Colorbar(fig[1, 2], p)
    hidespines!(ax)
    hidedecorations!(ax)
    ax.title = maketitle(p, title)

    return fig
end;

In [16]:
maketitle(p,"title")

UndefVarError: UndefVarError: `valuerange` not defined

In [15]:
plotr(m, :mx, "Biegemoment mx", (-17, 17), a3d=false)

KeyError: KeyError: key :post not found